Modules:

In [1]:
import numpy as np
import pandas as pd

Data cleaning and preparation:

In [ ]:
# Read df
df = pd.read_excel("data.xlsx")

### Yield ###
# Select date and yield from df
col_sel_d_yield = ["date_yield", "yield"]

# Create df for yield
df_d_yield = df[col_sel_d_yield].copy()

# Convert date column in df_yield to datetime
df_d_yield["date_yield"] = pd.to_datetime(df_d_yield["date_yield"])

# Set date as index
df_d_yield.set_index("date_yield", inplace=True)

# Monthly yield
df_m_yield =df_d_yield["yield"].resample("MS").mean()

# Reset index to create df 
df_m_yield = df_m_yield.reset_index()

# Set index
df_m_yield = df_m_yield.set_index("date_yield")

### SP500 ###
# Select date and yield from SP500
col_sel_sp500 = ["date_sp500", "sp500"]

# Create df for SP500
df_sp500 = df[col_sel_sp500].copy()

# Convert date column in df_sp500 to datetime
df_sp500["date_sp500"] = pd.to_datetime(df_sp500["date_sp500"])

# Remove missing values
df_sp500 = df_sp500.iloc[:660,:].copy()

# Set index
df_sp500 = df_sp500.set_index("date_sp500")

### PCE ### 
# Select date and yield from PCE
col_sel_pce = ["date_pce", "pce"]

# Create df for PCE
df_pce = df[col_sel_pce].copy()

# Convert date column in df_pce to datetime
df_pce["date_pce"] = pd.to_datetime(df_pce["date_pce"])

# Remove missing values
df_pce = df_pce.iloc[:793,:].copy()

# Set index
df_pce = df_pce.set_index("date_pce")

### CPI ###
# Select date and yield from CPI
col_sel_cpi = ["date_cpi", "cpi"]

# Create df for CPI
df_cpi = df[col_sel_cpi].copy()

# Convert date column in df_pce to datetime
df_cpi["date_cpi"] = pd.to_datetime(df_cpi["date_cpi"])

# Remove missing values
df_cpi = df_cpi.iloc[:937,:].copy()

# Set index
df_cpi = df_cpi.set_index("date_cpi")

### Final DataFrame ###
# Merge dfs
df_final = df_m_yield.join([df_sp500, df_pce, df_cpi], how="left")

# Change index name
df_final.index.name = 'date'

# Define your start and end dates
start_date = '1985-01-01'
end_date = '2025-01-01'

# Slice the DataFrame between the two dates
df_final = df_final.loc[start_date:end_date]

### Manipulate the final DataFrame ###
# Yield in pct.
df_final["yield"] = df_final["yield"]/100

### Final DataFrame Quarterly ###
# Filter for months January (1), April (4), July (7), and October (10)
df_final_qr = df_final[df_final.index.month.isin([1, 4, 7, 10])].copy()

# Ln returns nominal yield
df_final_qr["yield_nom_ret"] = np.log(1+df_final_qr["yield"])/4

# Ln returns all else
df_final_qr[["sp500_nom_ret", "pce_nom_ret", "cpi_chg"]] = np.log(df_final_qr.iloc[:, 1:4] / df_final_qr.iloc[:, 1:4].shift(1))

# Real returns
df_final_qr[["yield_real_ret", "sp500_real_ret", "pce_real_ret"]] = (
df_final_qr[["yield_nom_ret", "sp500_nom_ret", "pce_nom_ret"]]
    .subtract(df_final_qr["cpi_chg"], axis=0))



,yield,sp500,pce,cpi,yield_nom_ret,sp500_nom_ret,pce_nom_ret,cpi_chg,yield_real_ret,sp500_real_ret,pce_real_ret
date,,,,,,,,,,,
1985-01-01,0.080200,179.60,2618.8,105.700,0.019287,NaN,NaN,NaN,NaN,NaN,NaN
1985-04-01,0.082219,179.80,2659.5,107.000,0.019753,0.001113,0.015422,0.012224,0.007529,-0.011111,0.003198
1985-07-01,0.073164,190.90,2715.7,107.700,0.017653,0.059905,0.020912,0.006521,0.011132,0.053384,0.014391
1985-10-01,0.074009,189.80,2755.8,108.500,0.017850,-0.005779,0.014658,0.007401,0.010449,-0.013179,0.007257
1986-01-01,0.073019,211.80,2827.1,109.900,0.017619,0.109672,0.025544,0.012821,0.004798,0.096851,0.012723
...,...,...,...,...,...,...,...,...,...,...,...
2024-01-01,0.054538,4845.65,19308.5,309.794,0.013276,0.144474,0.012455,0.006935,0.006341,0.137539,0.005520
2024-04-01,0.054436,5035.69,19603.3,313.016,0.013252,0.038469,0.015153,0.010347,0.002905,0.028122,0.004806
2024-07-01,0.054318,5522.30,19866.3,313.566,0.013224,0.092244,0.013327,0.001756,0.011468,0.090488,0.011571


Model